Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML: https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Beautiful Soup

**Beautiful Soup** es una librería **Python** que permite extraer información de contenido en formato **HTML o XML**. Para usarla, es necesario especificar un **parser**, que es responsable de transformar un documento HTML o XML en un árbol complejo de objetos Python. Esto permite, por ejemplo, que podamos interactuar con los elementos de una página web como si estuviésemos utilizando las herramientas del desarrollador de un navegador.

A la hora de extraer información de una web, uno de los parsers más utilizado es el parser HTML de **lxml**. Precisamente, será el que utilicemos en este tutorial.

**Será necesario instalar las siguientes librerías** (si no las tienes ya):

        pip3 install beautifulsoup4 requests pandas

        pip3 install beautifulsoup4 

        pip3 install requests

        pip3 install pandas 

###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

Algunos ejemplos de Web Scraping utilizando Beautiful Soup:

https://j2logo.com/python/web-scraping-con-python-guia-inicio-beautifulsoup/

http://omz-software.com/pythonista/docs/ios/beautifulsoup_guide.html

https://towardsdatascience.com/top-5-beautiful-soup-functions-7bfe5a693482

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [6]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html
import numpy as np

## Caso 1: Scraping de un catálogo: Labirratorium

In [7]:
URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page='

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [8]:
# La web tiene 80 páginas con 12 cervezas listadas en cada página. 958

Hacemos la consulta (request) y creamos la SOPA inicial:

In [9]:
r = requests.get(URL)
soup = BeautifulSoup(r.text, "lxml")

In [10]:
type(soup)

bs4.BeautifulSoup

In [26]:
#soup esto nos traeria toda la web

In [28]:
# Guardamos lista de cervezas
cervezas_grid = soup.find_all(class_ = 'product-image')
len(cervezas_grid) #Numero de cervezas en esa pagina

12

In [31]:
# necesitamos acceder a cada una de las cervezas del grid:
lista_url = []
for cerveza in cervezas_grid:
    URL_cerveza = cerveza.find('a')['href']
    lista_url.append(URL_cerveza)
lista_url

['https://www.labirratorium.com/es/lambic/284-boon-kriek-2013.html',
 'https://www.labirratorium.com/es/alemania/225-stortebeker-schwarz-bier.html',
 'https://www.labirratorium.com/es/inicio/199-orval.html',
 'https://www.labirratorium.com/es/alemania/184-augustiner-lagerbier-hell.html',
 'https://www.labirratorium.com/es/inicio/183-schneider-eisbock.html',
 'https://www.labirratorium.com/es/inicio/181-schlenkerla-rauchbier-weizen.html',
 'https://www.labirratorium.com/es/inicio/173-samuel-adams-boston-lager.html',
 'https://www.labirratorium.com/es/inicio/165-laugar-epa.html',
 'https://www.labirratorium.com/es/inicio/82-westmalle-dubbel.html',
 'https://www.labirratorium.com/es/inicio/75-duchesse-de-bourgogne.html',
 'https://www.labirratorium.com/es/inicio/61-tripel-karmeliet33.html',
 'https://www.labirratorium.com/es/inicio/21-weihenstephaner-vitus.html']

In [34]:
# Hacemos un nuevo request para la primera cerveza: 
r = requests.get(lista_url[0])
soup_cerveza = BeautifulSoup(r.text, 'lxml')

In [36]:
# Nombre
nombre = soup_cerveza.find(class_ = 'h1 product-detail-name').text
print(nombre)

Boon Oude Kriek 37,5cl


In [40]:
# Precio
precio = soup_cerveza.find(class_ = 'current-price').find('span')['content']
precio

'7.15'

In [43]:
# Descripcion corta
descrp_corta = soup_cerveza.find(class_ = 'description-short').find('p').text
descrp_corta

'Lambic / Kriek'

In [44]:
# Descripción larga
descrp_larga = soup_cerveza.find(class_ = 'product-description').find('p').text
descrp_larga

'Cerveza de fermentación espontánea (Lambic) de 6.5% ABV sin filtrar ni pasteurizar de estilo Kriek, elaborada con cerezas naturales.'

In [46]:
#Imagen
imagen = soup_cerveza.find(class_ = 'product-cover').find(id = 'zoom_product')['src']
print(imagen)

https://www.labirratorium.com/19351-large_default/boon-kriek-2013.jpg


In [47]:
 # Brand
marca = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
marca

'Brouwerij F. Boon'

In [20]:
# Código de barras


In [50]:
# Features -- Crear un diccionario clave valor 
features_dic ={}
features = soup_cerveza.find(class_ = 'data-sheet')
features_dt = features.find_all('dt')
features_dd = features.find_all('dd')

for feature, value in zip(features_dt, features_dd):
    features_dic[feature.text] = value.text
features_dic

{'Estilo': 'KRIEK',
 'Origen': 'Bélgica',
 '% Alc.': '6.5\nALTO (6-9%)',
 'Otros ingredientes': 'Cerezas Naturales',
 'Volumen (cl)': '37.5 Cl',
 'Tipo Fermentación': 'Lambic (Fermentación espontánea o salvaje)',
 'Maltas': 'Cebada y Trigo',
 'IBU': '0-25 Amargor bajo',
 'Color': 'Rojiza',
 'Envase': 'Botella'}

In [22]:
# Creamos un Id único que os permita diferenciar cada entrada en la BBDD
id_cerv = 'lbt'+ str()

Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

In [54]:
# Agregamos a una lista

pages = np.arange(1,81)
count = 1
lista_cervezas = []

for page in pages:

    URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page='
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, "lxml")
    cervezas_grid = soup.find_all(class_ = 'product-image')

    count_beer = 1
    for cerveza in cervezas_grid:
        print('Cerveza {} de {}, pag {}/{}'.format(count_beer, len(cervezas_grid), page, len(pages)))
        URL_cerveza = cerveza.find('a')['href']
        r = requests.get(URL_cerveza)
        soup_cerveza = BeautifulSoup(r.text, 'lxml')

        id_cerv = 'lbt'+ str(count)
        
        nombre = soup_cerveza.find(class_ = 'h1 product-detail-name').text
        precio = soup_cerveza.find(class_ = 'current-price').find('span')['content']
        
        try:
            descrp_corta = soup_cerveza.find(class_ = 'description-short').find('p').text
        except:
            descrp_corta = None
        descrp_larga = soup_cerveza.find(class_ = 'product-description').find('p').text
        imagen = soup_cerveza.find(class_ = 'product-cover').find(id = 'zoom_product')['src']
        try:
            marca = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
        except:
            marca = None
        
        features_dic = {}
        features = soup_cerveza.find(class_ = 'data-sheet')
        try:
            features_dt = features.find_all('dt')
            features_dd = features.find_all('dd')
            for feature, value in zip(features_dt, features_dd):
                    features_dic[feature.text] = value.text
        except:
            features_dic = {}

        lista_cervezas = []
        lista_cervezas.append([
            ('id', id_cerv),
            ('name', nombre), 
            ('price', precio),
            ('descrp_corta', descrp_corta),
            ('image', imagen),
            ('brand', marca),
            ('features', features_dic)
            ])

        count += 1
        count_beer +=1

Cerveza 1 de 12, pag 1/80
Cerveza 2 de 12, pag 1/80
Cerveza 3 de 12, pag 1/80
Cerveza 4 de 12, pag 1/80
Cerveza 5 de 12, pag 1/80
Cerveza 6 de 12, pag 1/80
Cerveza 7 de 12, pag 1/80
Cerveza 8 de 12, pag 1/80
Cerveza 9 de 12, pag 1/80
Cerveza 10 de 12, pag 1/80
Cerveza 11 de 12, pag 1/80
Cerveza 12 de 12, pag 1/80
Cerveza 1 de 12, pag 2/80
Cerveza 2 de 12, pag 2/80
Cerveza 3 de 12, pag 2/80
Cerveza 4 de 12, pag 2/80
Cerveza 5 de 12, pag 2/80
Cerveza 6 de 12, pag 2/80
Cerveza 7 de 12, pag 2/80
Cerveza 8 de 12, pag 2/80
Cerveza 9 de 12, pag 2/80
Cerveza 10 de 12, pag 2/80
Cerveza 11 de 12, pag 2/80
Cerveza 12 de 12, pag 2/80
Cerveza 1 de 12, pag 3/80
Cerveza 2 de 12, pag 3/80
Cerveza 3 de 12, pag 3/80
Cerveza 4 de 12, pag 3/80
Cerveza 5 de 12, pag 3/80
Cerveza 6 de 12, pag 3/80
Cerveza 7 de 12, pag 3/80
Cerveza 8 de 12, pag 3/80
Cerveza 9 de 12, pag 3/80
Cerveza 10 de 12, pag 3/80
Cerveza 11 de 12, pag 3/80
Cerveza 12 de 12, pag 3/80
Cerveza 1 de 12, pag 4/80
Cerveza 2 de 12, pag 4/80
Cer

In [59]:
df = pd.DataFrame([[x[0][1],x[1][1],x[2][1],x[3][1],x[4][1],[5][1],x[6][1]] for x in lista_cervezas], columns = ['id','name', 'price','descrp_corta', 'image','brand','features'])
df.to_csv('df_labirratorium.csv', sep = ";")
df


IndexError: list index out of range

### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

In [ ]:
archivo imagem

### FBI: Top ten criminals

#### Queremos guardar las imágenes de cada fugitivo y que el nombre de cada archivo sea el nombre del fugitivo:

In [24]:
fbi_url = 'https://www.fbi.gov/wanted/topten'

